In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline uncomment this if you need to see the graph
pd.set_option('future.no_silent_downcasting', True) #to remove the future warning

class Model(nn.Module):
  #Inputs are sepal length, sepal width and petal length, petal width
  #There are 2 hidden layers h1 and h2 with 8 datapoints each
  #outputs are Iris Setosa , Iris Versicolour , Iris Virginica
  def __init__(self,inputFeature = 4,h1 = 8,h2 = 8, outputFeature = 3):
    super().__init__()
    self.fc1 = nn.Linear(inputFeature,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2,outputFeature)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

#manually selecting a seed
torch.manual_seed(41) #you can also change 41 to random for testing

#instance of class Model
model = Model()

#url of iris dataset csv file by netj from github
url = "https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv"
df = pd.read_csv(url)

#editing for comfortable processing
df['variety'] = df['variety'].replace('Setosa',0.0)
df['variety'] = df['variety'].replace('Versicolor',1.0)
df['variety'] = df['variety'].replace('Virginica',2.0)

# Train Test Split for set X,y
X = df.drop( 'variety', axis = 1)
y = df['variety'].astype(np.float64) #since typecasting of replace is decrypted

X = X.values
y = y.values

from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X , y , test_size= 0.2,random_state= 41) #training is 80% of total size

#converting X values to tensor float
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

#converting y values to tensor long
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

#this is for user convinience
def chkflwr(x):
  if x == 0:
    return 'Iris Setosa'
  elif x == 1:
   return 'Iris Versicolour'
  else:
    return 'Iris Virginica'
#setting the basis of model for entropyloss
criterion = nn.CrossEntropyLoss()
#choosing the optimizer (In this case Adam) and learning rate (lr)
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

#Model training
#set the number of Epochs
epochs = 100
losses = []
for i in range(epochs):
  y_prediction = model.forward(X_train)
  loss = criterion(y_prediction,y_train) # y_prediction vs y_train
  losses.append(loss.detach().numpy()) #keeping trach od the losses

  if i %10 == 0:
    print(f'Epoch {i} Loss {loss}')

  #backpropogation for optimizing the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

# You can see the graph for number of epochs vs losses below if needed
# plt.plot(range(epochs),losses)
# plt.xlabel('Epochs')
# plt.ylabel('Loss')

# Model training with test data set
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval,y_test)

correct = 0
with torch.no_grad():
  for i , data in enumerate(X_test):
    y_val = model.forward(data)
    print(f'{i+1} {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

    if y_val.argmax().item() == y_test[i]:
      correct += 1
print(f'\n{correct} out of {len(y_test)}')
print(f'\nAccuracy: {round(correct/len(y_test),3)}/1')
# save the nn model
torch.save(model.state_dict(),'model.pt')
#load the model
new_model = Model()
new_model.load_state_dict(torch.load('model.pt',weights_only=False))

#Test
new_dataPoint = torch.tensor([5.87,3.545,1.234,0.2])
with torch.no_grad():
  opt = chkflwr(new_model.forward(new_dataPoint).argmax().item())

print(f'\nThe Given Flower is {opt}')

Epoch 0 Loss 1.1318365335464478
Epoch 10 Loss 0.9658561944961548
Epoch 20 Loss 0.6789938807487488
Epoch 30 Loss 0.4004894196987152
Epoch 40 Loss 0.22509503364562988
Epoch 50 Loss 0.12207332998514175
Epoch 60 Loss 0.07481689006090164
Epoch 70 Loss 0.05451277643442154
Epoch 80 Loss 0.04437492415308952
Epoch 90 Loss 0.038379427045583725
1 tensor([-7.7212,  1.5826,  4.8685]) 	 2 	 2
2 tensor([-10.2316,   0.5146,   8.7767]) 	 2 	 2
3 tensor([-10.7282,   1.1893,   8.2408]) 	 2 	 2
4 tensor([-4.4743,  3.5419, -1.1241]) 	 1 	 1
5 tensor([-9.0954,  1.4687,  6.2899]) 	 2 	 2
6 tensor([-2.2803,  3.7077, -4.1196]) 	 1 	 1
7 tensor([-7.0740,  2.2070,  3.2966]) 	 2 	 2
8 tensor([-4.2271,  3.6553, -1.5467]) 	 1 	 1
9 tensor([-8.0707,  1.8280,  4.7897]) 	 2 	 2
10 tensor([-10.8583,   0.5051,   9.3761]) 	 2 	 2
11 tensor([-6.8676,  2.2165,  3.0859]) 	 2 	 2
12 tensor([  9.4382,   3.2638, -18.8717]) 	 0 	 0
13 tensor([  8.6037,   2.9307, -17.1502]) 	 0 	 0
14 tensor([-1.0594,  3.1702, -4.7116]) 	 1 	 1
